# Homework #10.2 (Jacobian Supplement):
|Author| Stanley A. Baronett|
|--|-------------------------------|
|Created | 11/3/2019|
|Updated | 11/6/2019|

## 18.3 Global Optimization II+
### Analytic Jacobian Optimization
Another way we can optimize our methods to find the ground state energy is to pass an explicit function for the _Jacobian_ of our objective function to SciPy's `minimize` function. Thus far, calls to SciPy's `minimize` require it to _numerically_ calculate the gradient of our objective function at some set of parameters (in our case, the series of $x, y,$ and $z$ coordinates of each atom). In some cases, e.g., when the Jacobian of the objective function cannot be expressed analytically, doing this numerically is the only way, but it comes at substantial computational cost.

$$ \mathbf J = \left[ \begin{array} { c c c } { \frac { \partial f } { \partial x _ { 1 } } } & { \cdots } & { \frac { \partial f } { \partial x _ { n } } } \end{array} \right],\tag{1}$$

where $x_i$ are each variable (parameter) of our objective function, $f$. Now, our objective function, $E$, for $N$ atoms, is

$$ E(x_1, y_1, z_1, ..., x_n, y_n, z_n) = \sum_{i=1}^{N-1}\sum_{j=2}^{N}V(r_{ij}), \tag{2}$$

where,

$$ r_{ij} = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2 + (z_i - z_j)^2}.\tag{3} $$

Let's calculate the first term of the of the Jacobian for $E$, $\mathbf J_1$:

$$ \mathbf J_1 = \frac{\partial E}{\partial x_1} = \sum_{j=2}^{N}\frac{\partial V(r_{1j})}{\partial r_{1j}}\frac{\partial r_{1j}}{\partial x_1}, \tag{4} $$

but,

$$ \frac{\partial V(r)}{\partial r} = 24 \left( \frac{1}{r^7} - \frac{2}{r^{13}} \right) \tag{5} $$

and,

$$ \frac{\partial r_{1j}}{\partial x_1} = \frac{x_1 - x_j}{r_{1j}} \tag{6}. $$

Thus, for the $i^{th}$ term in the Jacobian for $N$ atoms,

$$ \mathbf J_i = \frac{\partial E}{\partial x_i} = \sum_{j=i+1}^{N} \left( \frac{24}{r_{ij}^7} - \frac{48}{r_{ij}^{13}} \right)\frac{x_i - x_j}{r_{ij}}, \tag{4} $$

Thus, we can define a function to return the Jacobian for $N$ atoms.

In [ ]:
from Numba import jit

In [ ]:
@jit
def E_jac(pos):
    """
    ***
    """

Together with our originally defined functions...

In [1]:
@jit
def V(r):
    """
    Calculate and return the Lennard-Jones potential between two
    atoms as a function of separation distance.
    Input:
        r: interatomic separation distance.
    Output:
        Lennard-Jones potential with ε = σ = 1
    """
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def E(positions):
    """
    Calculate the total potential energy of N number of atoms using
    the Lennard-Jones potential.
    Input:
        positions: 3*N 1-D array which represents the atomic positions
                   (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    Output:
        E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    for i in range(N_atom - 1):
        for j in range(i + 1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1 - pos2)
            E += LJ(dist)
            
    return E

@jit
def init_pos(N, L=1):
    """
    Randomly initialize the 3-D position of N atoms within a cube
    of length L.
    Input:
        N: the number of atoms
        L: 3-D boundary of random sample (e.g., length of cube)
    Output:
        3*N 1-D array of atomic positions
        (e.g., [x0, y0, z0, x1, y2, z1, ..., xn, yn, zn])
    """
    return L*np.random.random_sample((N*3,))